# 🤖 AI-Powered Multi-Agent Banking Customer Churn Analytics Engine
## Use Case: Agentic Executive Intelligence Platform with Gemini-Powered Insights Q&A and Web Search Capabilities

### Kaggle 5-Day AI Agents Intensive Course with Google
### Multi-Agent Intelligent Insights Engine with Gemini-Powered Q&A on Bank Cystomer Churn Kaggle Dataset
### Version: 1.0
### Created by: Omar Chehab
### Date: 30-11-2025

## Project Overview
An advanced **agentic AI system** that transforms raw bank customer churn data into actionable executive insights. This project demonstrates a multi-agent architecture where specialized AI agents work autonomously to deliver intelligent analysis:

### 🤖 Agent Architecture Overview:

1. **Analytics Agent**: The primary intelligence engine that autonomously:
   - Performs multi-dimensional analysis on customer churn data (geographic, demographic, financial)
   - Generates executive summaries and risk assessments
   - Prepares comprehensive context for decision-making

2. **Web Search Agent**: An optional secondary agent that autonomously:
   - Searches for industry benchmarks and market trends when relevant
   - Enriches internal insights with external market intelligence
   - Filters queries to ensure relevance to churn analysis

3. **Gemini LLM Agent**: The orchestrator that:
   - Synthesizes insights from both Analytics and Web Search agents
   - Generates natural language responses tailored to executive audience
   - Adapts communication style based on context and data insights

**Key Agentic Capabilities**:
- ✅ Autonomous decision-making within defined domains
- ✅ Real-time data analysis and context generation
- ✅ Optional web search integration for market insights
- ✅ Conversational Q&A with multi-turn dialogue support
- ✅ Executive-level reporting with actionable recommendations

## 1. Importing Required Modules and Libraries
Import necessary libraries for data manipulation, API configuration, and Google ADK components for agent-based processing.

In [1]:
# Core Libraries
import os
import html
import re
from typing import Any, Dict, List

# Data Processing
import pandas as pd
import numpy as np

# Google AI & ADK
import google.generativeai as genai
from google.genai import types
from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner, InMemoryRunner
from google.adk.tools.tool_context import ToolContext
from google.adk.tools import AgentTool, FunctionTool, google_search

# Kaggle
from kaggle_secrets import UserSecretsClient

print("✅ All components imported successfully.")

✅ All components imported successfully.


## 2. API Configuration & Agent Setup
Configure Google Generative AI credentials from Kaggle Secrets and establish model parameters for the analytics agent.

In [2]:
# Load API Key from Kaggle Secrets
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Gemini API key setup complete.")
except ImportError:
    print("⚠️ Kaggle Secrets not available. Ensure you're in a Kaggle Notebook.")
except KeyError:
    print("🔑 Authentication Error: Add 'GOOGLE_API_KEY' to Kaggle secrets.")

# Agent Configuration
CONFIG = {
    "project": "",
    "model": "models/gemini-2.5-flash",
    "max_tokens": 2000,
    "temperature": 0.3,
    "version": "1.0"
}

print(f"\n{'='*60}")
print(f"{'AGENT CONFIGURATION':^60}")
print(f"{'='*60}")
for k, v in CONFIG.items():
    print(f"{k:.<25} {v}")
print(f"{'='*60}")

✅ Gemini API key setup complete.

                    AGENT CONFIGURATION                     
project.................. 
model.................... models/gemini-2.5-flash
max_tokens............... 2000
temperature.............. 0.3
version.................. 1.0


In [3]:
# HTTP Retry Configuration
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)
print("✅ Retry configuration defined.")

✅ Retry configuration defined.


In [4]:
# List all available LLM models
for model in genai.list_models():
    print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-l

## 3. Data Loading & Initial Exploration
Load the Bank Customer Churn Prediction dataset from Kaggle and perform initial data quality checks.

In [5]:
# Load Dataset
# Source: https://www.kaggle.com/datasets/saurabhbadole/bank-customer-churn-prediction-dataset
df = pd.read_csv("/kaggle/input/bank-customer-churn-prediction-dataset/Churn_Modelling.csv")

print(f"✅ Data loaded: {len(df)} rows")
print(f"📊 Columns: {df.columns.tolist()}")

✅ Data loaded: 10000 rows
📊 Columns: ['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']


In [6]:
# Checking sample output
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 4. Core Analytics Agent Implementation
Define the `AnalyticsAgent` class that performs multi-dimensional analysis on customer churn data:
- **Executive Summary**: Key metrics and overall statistics
- **Geographic Analysis**: Churn patterns by country/region
- **Demographic Analysis**: Age group and gender-based churn insights
- **Product Engagement**: Product count and member activity impact
- **Risk Segmentation**: Identification of high-risk customer groups
- **Financial Profiling**: Comparison of churned vs retained customers
- **Customer Lifetime Value Impact**: Revenue loss analysis
- **Gemini-Powered Q&A**: Natural language interface with optional web search

In [7]:
class AnalyticsAgent:
    """
    Analytics Agent for generating executive insights on Bank Customer Churn data.
    Powered by Google Gemini for intelligent Q&A with web search capability.
    """
    
    # Churn-related keywords for web search filtering
    CHURN_KEYWORDS = frozenset([
        'churn', 'retention', 'customer attrition', 'bank', 'finance',
        'customer behavior', 'risk', 'segment', 'loyalty', 'satisfaction',
        'industry', 'trend', 'benchmark', 'best practice', 'strategy',
        'retention rate', 'attrition rate', 'customer lifetime value',
        'clv', 'revenue', 'profitability', 'financial', 'demographic'
    ])
    
    def __init__(self, dataframe: pd.DataFrame):
        """Initialize the Analytics Agent with the dataset."""
        self.df = dataframe.copy()
        self.insights = {}
        self.chat_history = []
        
    def _calculate_churn_rate(self) -> float:
        """Calculate overall churn rate as percentage."""
        return (self.df['Exited'].sum() / len(self.df)) * 100
    
    def _calculate_percentage(self, column: str) -> float:
        """Calculate percentage of True values in a binary column."""
        return (self.df[column].sum() / len(self.df)) * 100
    
    def generate_executive_summary(self) -> Dict[str, Any]:
        """Generate a comprehensive executive summary with key metrics."""
        summary = {
            'total_customers': len(self.df),
            'churn_rate': self._calculate_churn_rate(),
            'avg_customer_age': self.df['Age'].mean(),
            'avg_account_balance': self.df['Balance'].mean(),
            'avg_estimated_salary': self.df['EstimatedSalary'].mean(),
            'active_member_rate': self._calculate_percentage('IsActiveMember'),
            'credit_card_holder_rate': self._calculate_percentage('HasCrCard'),
        }
        self.insights['executive_summary'] = summary
        return summary
    
    def analyze_churn_by_geography(self) -> pd.DataFrame:
        """Analyze churn rates across different geographies."""
        geo_analysis = self.df.groupby('Geography', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'EstimatedSalary': 'mean'
        }).round(2)
        geo_analysis.columns = ['Churned_Customers', 'Total_Customers', 'Churn_Rate', 
                                'Avg_Balance', 'Avg_Salary']
        geo_analysis['Churn_Rate'] *= 100
        self.insights['geography_analysis'] = geo_analysis
        return geo_analysis
    
    def analyze_churn_by_demographics(self) -> Dict[str, pd.DataFrame]:
        """Analyze churn patterns by demographic factors (age, gender)."""
        # Create age groups
        self.df['AgeGroup'] = pd.cut(
            self.df['Age'], 
            bins=[0, 30, 40, 50, 60, 100],
            labels=['<30', '30-40', '40-50', '50-60', '60+']
        )
        
        # Age analysis
        age_analysis = self.df.groupby('AgeGroup', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean'
        }).round(2)
        age_analysis.columns = ['Churned', 'Total', 'Churn_Rate', 'Avg_Balance']
        age_analysis['Churn_Rate'] *= 100
        
        # Gender analysis
        gender_analysis = self.df.groupby('Gender', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'CreditScore': 'mean'
        }).round(2)
        gender_analysis.columns = ['Churned', 'Total', 'Churn_Rate', 'Avg_Balance', 'Avg_CreditScore']
        gender_analysis['Churn_Rate'] *= 100
        
        demographics = {'age_analysis': age_analysis, 'gender_analysis': gender_analysis}
        self.insights['demographics'] = demographics
        return demographics
    
    def analyze_product_engagement(self) -> pd.DataFrame:
        """Analyze churn based on number of products and engagement metrics."""
        product_analysis = self.df.groupby('NumOfProducts', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'Tenure': 'mean',
            'IsActiveMember': 'mean'
        }).round(2)
        product_analysis.columns = ['Churned', 'Total', 'Churn_Rate', 
                                    'Avg_Balance', 'Avg_Tenure', 'Active_Rate']
        product_analysis['Churn_Rate'] *= 100
        product_analysis['Active_Rate'] *= 100
        self.insights['product_engagement'] = product_analysis
        return product_analysis
    
    def identify_high_risk_segments(self) -> pd.DataFrame:
        """Identify customer segments with highest churn risk."""
        segments = self.df.groupby(['Geography', 'Gender', 'IsActiveMember'], observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'Age': 'mean'
        }).round(2)
        segments.columns = ['Churned', 'Total', 'Churn_Rate', 'Avg_Balance', 'Avg_Age']
        segments['Churn_Rate'] *= 100
        
        high_risk = (segments[segments['Total'] >= 50]
                     .sort_values('Churn_Rate', ascending=False)
                     .head(10))
        self.insights['high_risk_segments'] = high_risk
        return high_risk
    
    def analyze_financial_profile(self) -> Dict[str, Any]:
        """Analyze financial characteristics of churned vs retained customers."""
        def get_profile(data: pd.DataFrame) -> Dict[str, float]:
            return {
                'avg_balance': data['Balance'].mean(),
                'median_balance': data['Balance'].median(),
                'avg_credit_score': data['CreditScore'].mean(),
                'avg_salary': data['EstimatedSalary'].mean(),
                'zero_balance_pct': (data['Balance'] == 0).mean() * 100
            }
        
        churned = self.df[self.df['Exited'] == 1]
        retained = self.df[self.df['Exited'] == 0]
        
        financial_profile = {
            'churned_customers': get_profile(churned),
            'retained_customers': get_profile(retained)
        }
        self.insights['financial_profile'] = financial_profile
        return financial_profile
    
    def calculate_customer_lifetime_value_impact(self) -> Dict[str, float]:
        """Calculate the financial impact of customer churn."""
        churned = self.df[self.df['Exited'] == 1]
        estimated_revenue_per_customer = 0.01
        
        impact = {
            'total_churned_customers': len(churned),
            'total_balance_lost': churned['Balance'].sum(),
            'avg_balance_per_churned_customer': churned['Balance'].mean(),
            'estimated_annual_revenue_loss': churned['Balance'].sum() * estimated_revenue_per_customer,
            'avg_tenure_of_churned': churned['Tenure'].mean(),
        }
        self.insights['clv_impact'] = impact
        return impact
    
    def get_all_insights(self) -> Dict[str, Any]:
        """Run all analyses and return comprehensive insights dictionary."""
        self.generate_executive_summary()
        self.analyze_churn_by_geography()
        self.analyze_churn_by_demographics()
        self.analyze_product_engagement()
        self.identify_high_risk_segments()
        self.analyze_financial_profile()
        self.calculate_customer_lifetime_value_impact()
        return self.insights
    
    # ============================================================================
    # GEMINI-POWERED Q&A FUNCTIONALITY WITH WEB SEARCH
    # ============================================================================
    
    def _prepare_context(self) -> str:
        """Prepare a comprehensive context string with all insights for Gemini."""
        if not self.insights:
            self.get_all_insights()
        
        lines = ["=== BANK CUSTOMER CHURN ANALYSIS DATA ===\n"]
        
        # Executive Summary
        lines.append("EXECUTIVE SUMMARY:")
        for key, value in self.insights['executive_summary'].items():
            lines.append(f"- {key.replace('_', ' ').title()}: {value:,.2f}")
        
        # Geography Analysis
        lines.append("\nCHURN BY GEOGRAPHY:")
        for geo, row in self.insights['geography_analysis'].iterrows():
            lines.append(
                f"- {geo}: {row['Churn_Rate']:.2f}% churn rate, "
                f"{int(row['Churned_Customers'])} of {int(row['Total_Customers'])} customers, "
                f"Avg Balance: ${row['Avg_Balance']:,.2f}"
            )
        
        # Demographics - Age
        lines.append("\nCHURN BY AGE GROUP:")
        for age_group, row in self.insights['demographics']['age_analysis'].iterrows():
            lines.append(
                f"- {age_group}: {row['Churn_Rate']:.2f}% churn rate, "
                f"{int(row['Churned'])} of {int(row['Total'])} customers"
            )
        
        # Demographics - Gender
        lines.append("\nCHURN BY GENDER:")
        for gender, row in self.insights['demographics']['gender_analysis'].iterrows():
            lines.append(
                f"- {gender}: {row['Churn_Rate']:.2f}% churn rate, "
                f"{int(row['Churned'])} of {int(row['Total'])} customers"
            )
        
        # Product Engagement
        lines.append("\nCHURN BY NUMBER OF PRODUCTS:")
        for num_products, row in self.insights['product_engagement'].iterrows():
            lines.append(
                f"- {int(num_products)} products: {row['Churn_Rate']:.2f}% churn rate, "
                f"{int(row['Churned'])} of {int(row['Total'])} customers, "
                f"Avg Tenure: {row['Avg_Tenure']:.1f} years, Active Rate: {row['Active_Rate']:.1f}%"
            )
        
        # High Risk Segments
        lines.append("\nTOP 5 HIGH-RISK SEGMENTS:")
        for idx, (segment, row) in enumerate(self.insights['high_risk_segments'].head(5).iterrows(), 1):
            geo, gender, is_active = segment
            active_status = "Active" if is_active == 1 else "Inactive"
            lines.append(
                f"{idx}. {geo} - {gender} - {active_status}: "
                f"{row['Churn_Rate']:.2f}% churn rate, {int(row['Total'])} customers"
            )
        
        # Financial Profile
        fp = self.insights['financial_profile']
        lines.append("\nFINANCIAL PROFILE COMPARISON:")
        for profile_type in ['churned_customers', 'retained_customers']:
            profile = fp[profile_type]
            label = profile_type.replace('_', ' ').title()
            lines.extend([
                f"{label}:",
                f"  - Avg Balance: ${profile['avg_balance']:,.2f}",
                f"  - Avg Credit Score: {profile['avg_credit_score']:.0f}",
                f"  - Avg Salary: ${profile['avg_salary']:,.2f}"
            ])
        
        # Financial Impact
        impact = self.insights['clv_impact']
        lines.extend([
            "\nFINANCIAL IMPACT:",
            f"- Total Churned Customers: {impact['total_churned_customers']:,}",
            f"- Total Balance Lost: ${impact['total_balance_lost']:,.2f}",
            f"- Estimated Annual Revenue Loss: ${impact['estimated_annual_revenue_loss']:,.2f}",
            f"- Avg Tenure of Churned: {impact['avg_tenure_of_churned']:.1f} years"
        ])
        
        return '\n'.join(lines)
    
    def _create_system_prompt(self) -> str:
        """Create the system prompt that defines the agent's role and behavior."""
        return """You are an expert Analytics Agent specializing in providing insights to executives based on bank customer churn analysis.

IMPORTANT FORMATTING RULES:
- Keep responses concise: 4-5 sentences per topic, 350-400 words maximum
- Always complete your sentences - never leave text unfinished
- Use plain text formatting, avoid complex markdown
- Use simple bullet points with - or numbers
- Do NOT use asterisks for emphasis
- EMOJI RULE: Only use emojis at the very START of a line or bullet point, NEVER in the middle or end of text
- NUMBER FORMAT: Show negative numbers with minus sign (e.g., -120) not parentheses (e.g., (120))

Your role is to answer executive questions about customer churn data with:
- Clear, concise, and actionable insights
- Data-driven responses based on the provided analysis
- Simple, plain language that anyone can understand (no jargon or technical terms)
- Specific numbers and percentages from the data
- Strategic recommendations when appropriate

When answering:
1. Reference specific data points from the analysis
2. Highlight key insights and patterns
3. Provide context and comparisons
4. End with 2-3 actionable recommendations
5. Be direct - no unnecessary preamble
6. Use bullet points and clear structure
7. If asked beyond scope, mention you can only answer churn-related questions
8. Only place emojis at the very beginning of lines, never within sentence text
9. Keep each response to 4-5 clear sentences per section

The data context below contains all the churn analysis results you should reference."""
    
    def _get_api_key(self) -> str:
        """Get API key from Kaggle Secrets."""
        return UserSecretsClient().get_secret("GOOGLE_API_KEY")
    
    def model_config(self, system_prompt: str, user_prompt: str) -> str:
        """Configure and call Gemini model with the given prompts."""
        try:
            api_key = self._get_api_key()
        except Exception:
            return "[Simulated LLM: GOOGLE_API_KEY secret not accessible in this environment.]"
        
        try:
            genai.configure(api_key=api_key)
            model = genai.GenerativeModel(
                "gemini-2.5-flash",
                generation_config={"max_output_tokens": 8192, "temperature": 0.7}
            )
            response = model.generate_content(f"{system_prompt}\n\n{user_prompt}")
            return response.text
        except Exception as e:
            return f"[Simulated LLM: Gemini unreachable → {e}]"
    
    def _is_churn_related_query(self, question: str) -> bool:
        """Determine if a query is relevant to customer churn analysis."""
        question_lower = question.lower()
        return any(keyword in question_lower for keyword in self.CHURN_KEYWORDS)
    
    def _search_external_info(self, question: str) -> str:
        """Search for external information relevant to the churn analysis query."""
        if not self._is_churn_related_query(question):
            return ""
        
        try:
            api_key = self._get_api_key()
            genai.configure(api_key=api_key)
            
            model = genai.GenerativeModel(
                "gemini-2.5-flash",
                system_instruction="You are a helpful research assistant. Search the web for current information and provide accurate, up-to-date facts."
            )
            
            prompt = f"""Search the web for current information about: customer churn banking {question}
            
Provide a concise summary with:
1. Key industry benchmarks or trends related to this question
2. Best practices or strategies
3. Relevant statistics or data points

Keep the summary brief (2-3 sentences) and directly relevant to bank customer churn analysis."""
            
            response = model.generate_content(
                prompt,
                generation_config=genai.GenerationConfig(temperature=0.7)
            )
            
            if response and response.text:
                return f"📱 EXTERNAL INSIGHTS FROM WEB SEARCH:\n{response.text}"
            return ""
            
        except Exception:
            print("ℹ️ Note: Web search not available in this environment")
            return ""
    
    def _ensure_insights_loaded(self):
        """Ensure insights are generated before Q&A."""
        if not self.insights:
            print("📊 Analyzing data... Please wait...")
            self.get_all_insights()
            print("✅ Analysis complete!\n")
    
    def ask_with_search(self, question: str) -> str:
        """Ask a question with external web search capability."""
        self._ensure_insights_loaded()
        self.chat_history.append({'role': 'user', 'content': question})
        
        print("🔍 Searching for external insights...")
        external_info = self._search_external_info(question)
        
        context = self._prepare_context()
        system_prompt = self._create_system_prompt()
        
        if external_info:
            user_prompt = f"DATA CONTEXT:\n{context}\n\n{external_info}\n\nEXECUTIVE QUESTION:\n{question}\n\nCombine internal data insights with the external research to provide a comprehensive answer:"
        else:
            user_prompt = f"DATA CONTEXT:\n{context}\n\nEXECUTIVE QUESTION:\n{question}\n\nProvide a clear, data-driven answer:"
        
        answer = self.model_config(system_prompt, user_prompt)
        self.chat_history.append({'role': 'agent', 'content': answer})
        return answer
    
    def ask(self, question: str) -> str:
        """Main Q&A interface. Ask the agent any question about the churn data."""
        self._ensure_insights_loaded()
        self.chat_history.append({'role': 'user', 'content': question})
        
        context = self._prepare_context()
        system_prompt = self._create_system_prompt()
        user_prompt = f"DATA CONTEXT:\n{context}\n\nEXECUTIVE QUESTION:\n{question}\n\nProvide a clear, data-driven answer:"
        
        answer = self.model_config(system_prompt, user_prompt)
        self.chat_history.append({'role': 'agent', 'content': answer})
        return answer
    
    def start_chat(self):
        """Start an interactive chat session (for Jupyter notebooks or console)."""
        print("=" * 80)
        print("🤖 ANALYTICS AGENT - EXECUTIVE Q&A SESSION (Powered by Gemini)")
        print("=" * 80)
        print("\nHello! I'm your AI-powered Analytics Agent. I can answer questions about")
        print("customer churn using advanced language understanding.")
        print("\nType 'quit', 'exit', or 'bye' to end the session.\n")
        print("-" * 80)
        
        self._ensure_insights_loaded()
        
        while True:
            try:
                question = input("\n💼 Executive: ").strip()
                
                if question.lower() in ['quit', 'exit', 'bye', 'q']:
                    print("\n👋 Thank you for using Analytics Agent. Goodbye!")
                    break
                
                if not question:
                    continue
                
                print("\n🤖 Agent: [Thinking...]\n")
                print(self.ask(question))
                print("\n" + "-" * 80)
                
            except KeyboardInterrupt:
                print("\n\n👋 Session ended. Goodbye!")
                break
            except Exception as e:
                print(f"\n❌ Error: {e}")
                print("Please try rephrasing your question.\n")
    
    def get_chat_history(self) -> List[Dict[str, str]]:
        """Return the chat history."""
        return self.chat_history
    
    def clear_chat_history(self):
        """Clear the chat history."""
        self.chat_history = []
        print("✅ Chat history cleared.")
    
    def print_executive_report(self):
        """Print a formatted executive report to console."""
        insights = self.get_all_insights()
        
        print("=" * 80)
        print("EXECUTIVE INSIGHTS REPORT - BANK CUSTOMER CHURN ANALYSIS")
        print("=" * 80)
        
        print("\n📈 EXECUTIVE SUMMARY:")
        print("-" * 80)
        for key, value in insights['executive_summary'].items():
            key_formatted = key.replace('_', ' ').title()
            fmt = f"{value:,}" if isinstance(value, int) else f"{value:,.2f}"
            print(f"  {key_formatted}: {fmt}")
        
        print("\n\n🌍 CHURN BY GEOGRAPHY:")
        print("-" * 80)
        print(insights['geography_analysis'])
        
        print("\n\n📦 PRODUCT ENGAGEMENT:")
        print("-" * 80)
        print(insights['product_engagement'])
        
        print("\n\n⚠️ TOP 5 HIGH-RISK SEGMENTS:")
        print("-" * 80)
        print(insights['high_risk_segments'].head())
        
        print("\n" + "=" * 80)

print("✅ AnalyticsAgent class loaded successfully!")

✅ AnalyticsAgent class loaded successfully!


In [8]:
# Create the Analytics Agent
agent = AnalyticsAgent(df)

print("✅ Agent initialized and ready!")

✅ Agent initialized and ready!


## 5. Agent Initialization & General Q&A Demo
Initialize the analytics agent with the churn dataset and demonstrate its Q&A capabilities with foundational questions.

In [9]:
agent.ask("What can you help with? What type of dataset can you support me with?")

📊 Analyzing data... Please wait...
✅ Analysis complete!



'I am an expert Analytics Agent specializing in bank customer churn analysis. I can provide clear, data-driven insights and actionable recommendations based on the provided bank customer churn analysis data. My expertise covers identifying high-risk segments, understanding churn drivers like geography, age, gender, and product usage, and assessing the financial impact of churn.\n\nThe type of dataset I can support you with is specifically the comprehensive bank customer churn analysis data provided. This includes details on churn rates by various demographics, product usage, financial profiles, and the overall financial impact of customer attrition. I am equipped to analyze this specific dataset to answer executive questions about customer retention.'

## 6. Interactive Q&A Demonstrations
Ask the agent specific questions about churn patterns, demographics, and business insights. These demonstrations showcase different analysis dimensions.

In [10]:
agent.ask("What's our overall churn rate?")

'Our overall customer churn rate is 20.37%. This means 2,037 out of our 10,000 total customers have exited the bank. This churn has led to a significant loss of $185,588,094.63 in total customer balances and an estimated annual revenue loss of $1,855,880.95. Addressing this rate is critical to maintaining our customer base and financial stability.\n\nHere are some initial recommendations:\n- Prioritize identifying and engaging our high-risk customer segments, such as those in Germany or with 3-4 products, to prevent further churn.\n- Develop targeted retention programs that offer incentives or enhanced services to customers showing early signs of dissatisfaction.\n- 🎯 Conduct follow-up surveys with churned customers to gain deeper insights into their reasons for leaving and inform future strategy.'

## 7. Interactive Visual Chatbot Interface
Create a user-friendly web-based chatbot UI using Jupyter Widgets with the following features:
- **Real-time chat interface** with message history
- **Quick question buttons** for common inquiries
- **Web search toggle** for enriched insights with external data
- **Responsive design** with color-coded messages
- **Gemini-powered responses** formatted for executive consumption

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

class VisualChatbot:
    """Visual chatbot using Jupyter widgets with web search capability."""
    
    # HTML Templates
    WELCOME_TEMPLATE = """
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
        <h2 style="margin: 0; color: white;">🤖 Bank Customer Churn Analytics Agent</h2>
        <p style="margin: 10px 0; color: white; font-size: 16px;">Powered by Google Gemini AI with Optional Web Search</p>
        <p style="font-size: 14px; opacity: 0.95; color: white;">Ask me anything about customer churn patterns and recommendations!</p>
    </div>
    """
    
    USER_MSG_TEMPLATE = """
    <div style="background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #2196f3; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        <strong style="color: #1565c0; font-size: 14px;">💼 You:</strong>
        <p style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1.6;">{message}</p>
    </div>
    """
    
    AGENT_MSG_TEMPLATE = """
    <div style="background: #f5f5f5; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #667eea; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        <strong style="color: #5e35b1; font-size: 14px;">🤖 Agent:</strong>
        <div style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1.8;">{message}</div>
    </div>
    """
    
    ERROR_MSG_TEMPLATE = """
    <div style="background: #ffebee; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #f44336; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        <strong style="color: #c62828; font-size: 14px;">❌ Error:</strong>
        <p style="margin: 8px 0 0 0; color: #212121; font-size: 14px;">{message}</p>
    </div>
    """
    
    THINKING_TEMPLATE = """
    <div style="background: #fff9c4; padding: 10px; border-radius: 8px; margin: 10px 0; text-align: center;">
        <span style="color: #f57c00;">🤖 Agent is thinking {search_status}...</span>
    </div>
    """
    
    # Emoji pattern for cleanup
    EMOJI_PATTERN = re.compile(r'[\U0001F300-\U0001F9FF\U00002600-\U000027BF\U0001FA00-\U0001FAFF]')
    
    def __init__(self, agent):
        self.agent = agent
        self.chat_history = []
        self.use_web_search = False
        self.setup_gemini()
        self.create_ui()
    
    def format_response(self, text: str) -> str:
        """Format agent response text for HTML display."""
        # Clean emojis from middle of sentences
        lines = text.split('\n')
        cleaned_lines = []
        
        for line in lines:
            stripped = line.lstrip()
            leading_space = line[:len(line) - len(stripped)]
            
            # Check if line starts with emoji
            start_pattern = r'^([-*•]?\s*\d*\.?\s*)(' + self.EMOJI_PATTERN.pattern + r'+\s*)'
            match = re.match(start_pattern, stripped)
            
            if match:
                prefix = match.group(0)
                rest = self.EMOJI_PATTERN.sub('', stripped[len(match.group(0)):])
                cleaned_lines.append(leading_space + prefix + rest)
            else:
                cleaned_lines.append(leading_space + self.EMOJI_PATTERN.sub('', stripped))
        
        text = '\n'.join(cleaned_lines)
        
        # Convert parentheses-style negative numbers to minus sign
        text = re.sub(r'\((\$?[\d,]+\.?\d*)\)', r'-\1', text)
        
        # Clean orphaned markdown
        text = re.sub(r'\*+\s*$', '', text)
        if text.count('**') % 2 != 0:
            last_pos = text.rfind('**')
            if last_pos != -1:
                text = text[:last_pos] + text[last_pos+2:]
        
        # Convert markdown to placeholders
        text = re.sub(r'\*\*(.+?)\*\*', r'__BOLD_START__\1__BOLD_END__', text)
        text = text.replace('**', '')
        text = re.sub(r'\*([^*\n]+?)\*', r'__ITALIC_START__\1__ITALIC_END__', text)
        text = re.sub(r'\*', '', text)
        text = re.sub(r'^### (.+)$', r'__H3_START__\1__H3_END__', text, flags=re.MULTILINE)
        text = re.sub(r'^## (.+)$', r'__H2_START__\1__H2_END__', text, flags=re.MULTILINE)
        text = re.sub(r'^# (.+)$', r'__H1_START__\1__H1_END__', text, flags=re.MULTILINE)
        text = re.sub(r'^- (.+)$', r'__BULLET__\1', text, flags=re.MULTILINE)
        text = re.sub(r'^(\d+)\. (.+)$', r'__NUM_\1__\2', text, flags=re.MULTILINE)
        
        # Escape HTML
        text = html.escape(text)
        
        # Convert placeholders to HTML
        replacements = {
            '__BOLD_START__': '<strong>', '__BOLD_END__': '</strong>',
            '__ITALIC_START__': '<em>', '__ITALIC_END__': '</em>',
            '__H1_START__': '<h3 style="color: #333; margin: 15px 0 10px 0;">', '__H1_END__': '</h3>',
            '__H2_START__': '<h4 style="color: #444; margin: 12px 0 8px 0;">', '__H2_END__': '</h4>',
            '__H3_START__': '<h5 style="color: #555; margin: 10px 0 6px 0;">', '__H3_END__': '</h5>',
            '__BULLET__': '&nbsp;&nbsp;• ',
        }
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Numbered lists
        for i in range(1, 20):
            text = text.replace(f'__NUM_{i}__', f'&nbsp;&nbsp;{i}. ')
        
        # Newlines and cleanup
        text = text.replace('\n', '<br>')
        text = re.sub(r'(<br>){3,}', '<br><br>', text)
        
        return text
    
    def setup_gemini(self):
        """Setup Gemini API."""
        try:
            api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
            genai.configure(api_key=api_key)
            
            model = genai.GenerativeModel(
                model_name="gemini-2.5-flash",
                generation_config={"temperature": 0.7, "top_p": 0.95, "max_output_tokens": 8192}
            )
            
            context = self.agent._prepare_context()
            system_instruction = f"""You are an expert Analytics Agent specializing in bank customer churn analysis.

{context}

Provide clear, data-driven insights with specific numbers and actionable recommendations."""
            
            self.chat_session = model.start_chat(history=[])
            self.chat_session.send_message(system_instruction)
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def create_ui(self):
        """Create widget-based UI."""
        # Chat output area
        self.chat_output = widgets.Output(
            layout=widgets.Layout(
                min_height='500px', max_height='1000px',
                border='2px solid #667eea', padding='15px',
                overflow_y='auto', background_color='#ffffff', flex='1 1 auto'
            )
        )
        
        # Input box
        self.input_box = widgets.Text(
            placeholder='Type your question here...',
            layout=widgets.Layout(width='65%'),
            style={'description_width': 'initial'}
        )
        
        # Buttons
        self.send_button = widgets.Button(
            description='Send 📤', button_style='primary',
            layout=widgets.Layout(width='17%', height='38px')
        )
        
        self.search_toggle = widgets.ToggleButton(
            value=False, description='🔍 Search Web: OFF', button_style='danger',
            layout=widgets.Layout(width='17%', height='38px'),
            tooltip='Enable external web search for churn-related queries'
        )
        
        self.clear_button = widgets.Button(
            description='Clear Chat 🗑️', button_style='warning',
            layout=widgets.Layout(width='100%', margin='10px 0')
        )
        
        # Quick question buttons
        button_layout = widgets.Layout(width='auto', min_width='220px', padding='5px 15px', margin='5px')
        quick_questions = [
            ("📊 What's our churn rate?", "What's our overall churn rate?"),
            ("⚠️ High-risk segments?", "Who are our high-risk customer segments?"),
            ("💡 Recommendations?", "What are your top recommendations to reduce churn?"),
            ("💰 Financial impact?", "What's the financial impact of churn?"),
            ("🌍 Geography analysis?", "Which geography has the highest churn?"),
            ("📋 Executive summary?", "Give me an executive summary"),
        ]
        
        self.quick_buttons = [
            widgets.Button(description=label, button_style='info', layout=button_layout)
            for label, _ in quick_questions
        ]
        
        # Event handlers
        self.send_button.on_click(self.on_send)
        self.input_box.on_submit(self.on_send)
        self.clear_button.on_click(self.on_clear)
        self.search_toggle.observe(self.on_search_toggle, names='value')
        
        for btn, (_, question) in zip(self.quick_buttons, quick_questions):
            btn.on_click(lambda b, q=question: self.send_quick_question(q))
        
        # Display welcome message
        with self.chat_output:
            display(HTML(self.WELCOME_TEMPLATE))
    
    def on_search_toggle(self, change):
        """Handle web search toggle."""
        self.use_web_search = change['new']
        if self.use_web_search:
            self.search_toggle.description = '🔍 Search Web: ON'
            self.search_toggle.button_style = 'success'
        else:
            self.search_toggle.description = '🔍 Search Web: OFF'
            self.search_toggle.button_style = 'danger'
        print(f"Web Search {'🟢 ON' if self.use_web_search else '🔴 OFF'}")
    
    def display(self):
        """Display the chat interface."""
        header = widgets.HTML("""
        <div style="margin-bottom: 15px; padding: 10px; background: #f8f9fa; border-radius: 8px; border-left: 4px solid #667eea;">
            <h3 style="margin: 0; color: #333;">💡 Quick Questions (Click to Ask):</h3>
        </div>
        """)
        
        quick_buttons_row1 = widgets.HBox(
            self.quick_buttons[:3],
            layout=widgets.Layout(justify_content='flex-start', margin='5px 0')
        )
        quick_buttons_row2 = widgets.HBox(
            self.quick_buttons[3:],
            layout=widgets.Layout(justify_content='flex-start', margin='5px 0')
        )
        
        chat_label = widgets.HTML("""
        <div style='margin: 20px 0 10px 0; padding: 10px; background: #f8f9fa; border-radius: 8px; border-left: 4px solid #667eea;'>
            <strong style="color: #333;">💬 Chat:</strong>
        </div>
        """)
        
        chat_area = widgets.VBox(
            [chat_label, self.chat_output],
            layout=widgets.Layout(flex='1 1 auto', min_height='350px')
        )
        
        input_section = widgets.VBox([
            widgets.HBox(
                [self.input_box, self.send_button, self.search_toggle],
                layout=widgets.Layout(width='100%', margin='10px 0 0 0', align_items='center')
            ),
            self.clear_button
        ], layout=widgets.Layout(flex='0 0 auto'))
        
        ui = widgets.VBox(
            [header, widgets.VBox([quick_buttons_row1, quick_buttons_row2]), chat_area, input_section],
            layout=widgets.Layout(display='flex', flex_flow='column', height='900px')
        )
        display(ui)
    
    def on_send(self, b):
        """Handle send button click."""
        message = self.input_box.value.strip()
        if message:
            self.send_message(message)
            self.input_box.value = ''
    
    def send_quick_question(self, question):
        """Send a quick question."""
        self.send_message(question)
    
    def _display_message(self, user_msg: str, agent_msg: str):
        """Display a user message and agent response."""
        with self.chat_output:
            display(HTML(self.USER_MSG_TEMPLATE.format(message=html.escape(user_msg))))
            display(HTML(self.AGENT_MSG_TEMPLATE.format(message=self.format_response(agent_msg))))
    
    def _redisplay_chat_history(self):
        """Redisplay all chat history."""
        with self.chat_output:
            display(HTML(self.WELCOME_TEMPLATE))
        for item in self.chat_history:
            self._display_message(item['user'], item['agent'])
    
    def send_message(self, message: str):
        """Send message and display response."""
        # Display user message
        with self.chat_output:
            display(HTML(self.USER_MSG_TEMPLATE.format(message=html.escape(message))))
        
        # Show thinking indicator
        with self.chat_output:
            search_status = "🔍 with Web Search" if self.use_web_search else ""
            display(HTML(self.THINKING_TEMPLATE.format(search_status=search_status)))
        
        try:
            # Get response
            response_text = (self.agent.ask_with_search(message) 
                           if self.use_web_search else self.agent.ask(message))
            
            # Clear and redisplay
            self.chat_output.clear_output(wait=True)
            self._redisplay_chat_history()
            self._display_message(message, response_text)
            
            # Store in history
            self.chat_history.append({"user": message, "agent": response_text})
            
        except Exception as e:
            self.chat_output.clear_output(wait=True)
            self._redisplay_chat_history()
            with self.chat_output:
                display(HTML(self.USER_MSG_TEMPLATE.format(message=html.escape(message))))
                display(HTML(self.ERROR_MSG_TEMPLATE.format(message=html.escape(str(e)))))
    
    def on_clear(self, b):
        """Clear chat history."""
        self.chat_output.clear_output()
        self.chat_history = []
        with self.chat_output:
            display(HTML(self.WELCOME_TEMPLATE))
            display(HTML("""
            <div style="background: #e8f5e9; padding: 15px; border-radius: 8px; text-align: center; border: 2px solid #4caf50;">
                <span style="color: #2e7d32; font-size: 16px; font-weight: bold;">✅ Chat cleared! Ask me anything about customer churn.</span>
            </div>
            """))


def create_visual_chatbot(agent):
    """Create and display visual chatbot."""
    chatbot = VisualChatbot(agent)
    chatbot.display()
    return chatbot

print("✅ Visual chatbot loaded with web search capability!")

✅ Visual chatbot loaded with web search capability!


## 8. Launch Interactive Chatbot
Instantiate and display the visual chatbot for interactive exploration of churn analysis insights.

In [12]:
# Create and display the improved chatbot
visual_chatbot = create_visual_chatbot(agent)

/tmp/ipykernel_13/482361943.py:199: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.input_box.on_submit(self.on_send)


## 9. Advanced Q&A Examples (Uncomment to Explore)
Below are commented examples of advanced queries you can uncomment to explore:
- **Geographic Analysis**: Compare churn patterns across different countries and regions
- **Product & Engagement**: Analyze how product holdings and member activity affect churn
- **Risk & Segments**: Identify highest-risk customer profiles and strategic priorities
- **Financial Impact**: Calculate revenue losses and ROI of retention strategies
- **Recommendations**: Get data-driven action items and best practices for retention

Uncomment any question and run the cell to see the agent's detailed analysis and insights.

### Geographic Analysis
Analyze churn patterns across different countries and regions to identify geographic risk factors.

- *"Which countries have the highest churn?"*
- *"Why is Germany churning more than other countries?"*

### Product & Engagement
Understand how product holdings and member activity levels impact churn behavior.

- *"How do product holdings affect churn?"*
- *"What about active vs inactive members?"*

### Risk & Segments
Identify highest-risk customer profiles and strategic priorities for retention efforts.

- *"Who are our highest-risk customers?"*
- *"What customer segments should we prioritize?"*

### Financial Impact
Calculate revenue losses and assess the ROI of retention strategies.

- *"What's the financial impact of churn?"*
- *"How much revenue are we losing?"*
- *"If we reduce churn by 15%, what's the savings?"*

### Recommendations
Get data-driven action items and best practices for reducing customer churn.

- *"What should we do to reduce churn?"*
- *"Give me your top 5 action items"*
- *"What's our best retention strategy?"*

### Complex Questions
Multi-dimensional analysis combining geographic, demographic, and product factors with external research.

- *"Compare Germany vs France churn patterns"*
- *"Why do customers with more products churn more?"*
- *"What's the profile of a typical churned customer?"*
- *"Can you use Google Search to check 3 churn trends in Germany? What could be driving churn?"*
- *"Look up reasons that could impact customers to churn in Spain. Which competitors would be attracting them?"*